##  GEE STAC Catalog

Use `pystac_client` package to query a STAC catalog. You can install via `conda install -c pystac-client`.

In [2]:
from pystac_client import Client
import pandas as pd

Create a STAC Catalog object and get all collections.

In [3]:
catalog = Client.open('https://earthengine-stac.storage.googleapis.com/catalog/catalog.json')
collections = catalog.get_all_collections()

The `collections` is a generator. Convert it to a list of [STAC Collection](https://pystac.readthedocs.io/en/stable/api/collection.html) objects.  *This can take some time to complete*

In [ ]:
items = list(collections)

Extract the relevant info into a list of dictionaries.

In [ ]:
datasets = []
for item in items:
    datasets.append({
        'id': item.id,
        'title': item.title,
        'description':item.description
    })

Remove all deprecated collections.

In [ ]:
datasets = [d for d in datasets if 'deprecated' in d['title']]

Create a Pandas DataFrame.

In [ ]:
df = pd.DataFrame(datasets)
df

Save to Excel.

In [ ]:
df.to_excel('ee_datasets.xlsx', index=False)